# Initial Model Testing

## Imports

In [ ]:
import string

import tensorflow as tf

from fun_with_words import data as fd

## Get the data

In [ ]:
data = fd.get_full_dataset()

In [ ]:
len(data)

## Make basic layers needed for model

In [ ]:
class Tokenizer(tf.keras.layers.Layer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    def call(self, inputs):
        return tf.strings.split(inputs)
    def compute_output_shape(self, input_shapes):
        return (*input_shapes, None)

class CharacterSplitter(tf.keras.layers.Layer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    def call(self, inputs):
        return tf.strings.bytes_split(inputs)
    def compute_output_shape(self, input_shapes):
        return (*input_shapes, None)


In [ ]:
t = Tokenizer()
c = CharacterSplitter()

In [ ]:
inp = tf.keras.Input(shape=(), dtype=tf.string)
out = c(t(inp))
model = tf.keras.Model(inp, out)

In [ ]:
model.summary()

In [ ]:
VOCAB = string.ascii_letters + string.digits + string.punctuation

class Encoder(tf.keras.layers.Layer):
    def __init__(self, vocabulary=VOCAB, **kwargs):
        super().__init__(**kwargs)
        self.vocabulary = vocabulary
        self.lookup = tf.lookup.StaticHashTable(
            initializer=tf.lookup.KeyValueTensorInitializer(
                
            )
        )

    def compute_output_shape(self, input_shape):
        return input_shape

    def call(self, inputs):
        return self.lookup.lookup(inputs)

    def get_config(self):
        cfg = super().get_config()
        cfg.update(dict(vocabulary=self.vocabulary))
        return cfg

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [ ]:
e = Encoder()